# ComCamSim 2024 DIA Sprint
Authors: Michael Wood-Vasey <wmwv@pitt.edu>  
Last Verified to Run: 2024-05-14  

DIA Sprint.  This Notebook is to explore and figure out how to calculate the Metrics and visualizations we want to look at dipole orientation across an image and check against other things of interest.  Some key new quantities calculated are

"ip_diffim_DipoleFit_separation"  
"ip_diffim_DipoleFit_orientation"

Let's see what we can learn about the orientation on the sky.

1. [x] Count dipoles and plot on image
3. [ ] Look at pixel stamps of dipoles
4. [ ] Spatially plot dipole orientation
5. [ ] Plot w.r.t. Parallactic Angle.

Run at RSP on USDF.  Run using `d_2024_05_15` on a Large container (because we need the memory).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler
from lsst.daf.butler import DatasetNotFoundError
from lsst.pipe.base import Instrument
from lsst.ap.association.transformDiaSourceCatalog import UnpackApdbFlags
import lsst.afw.display as afwDisplay
import lsst.display.astrowidgets
from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, MinMaxInterval, LogStretch
from astropy.table import Table, join

afwDisplay.setDefaultBackend('matplotlib')

Key orientation information: repo, collection, dataset types.

In [ ]:
repo = "/repo/embargo"
collection = "u/elhoward/DM-44138/LSSTComCamSim"
collections = [collection, "LSSTComCamSim/templates", "LSSTComCamSim/defaults"]
instrument = "LSSTComCamSim"

Dataset Types we'll be interested in  
`postISRCCD`  (I don't think there's a `calexp` in prompt processing)  
`goodSeeingDiff_differenceExp`  
`goodSeeingDiff_diaSrc`  (or `goodSeeingDiff_diaSrcTable` depending on the schema you want.)

In [ ]:
butler = Butler(repo, collections=collections, instrument=instrument)

In [ ]:
registry = butler.registry
for ref in registry.queryDatasets('goodSeeingDiff_differenceExp'):
    print(ref.dataId)

In [ ]:
data_id = {"detector": 0, "exposure": 7024040400021}
# An alternate data_id to look at
# data_id = ["detector": 5, "exposure": 7024040300440}
data_id["visit"] = data_id["exposure"]

In [ ]:
science = butler.get("initial_pvi", dataId=data_id)
diff = butler.get("goodSeeingDiff_differenceExp", dataId=data_id)
template = butler.get("goodSeeingDiff_templateExp", dataId=data_id)

In [ ]:
# Catalog from raw pipeline output is a `lsst.afw.table.SourceCatalog` and includes 100 blank "sky" sources
full_cat = butler.get("goodSeeingDiff_diaSrc", dataId=data_id )
# The diaSrcTable has things in SDMified form and will be a `DataFrame` and has removed the blank sky sources.
filtered_transformed_cat = butler.get("goodSeeingDiff_diaSrcTable", dataId=data_id )

In [ ]:
dia_sources = full_cat[full_cat["sky_source"] == False]
sky_sources = full_cat[full_cat["sky_source"] == True]

# Deep copy to get a new contiguous catalog
dipoles = dia_sources[dia_sources["ip_diffim_DipoleFit_flag_classification"]].copy(deep=True)

In [ ]:
# filtered_transformed_cat.columns

In [ ]:
print(f"{len(filtered_transformed_cat)} total detections.")
print(f"{len(dipoles)} detections classified as dipoles.")

In [ ]:
# Plot direction of arrows on sky
plt.quiver(dipoles["base_SdssCentroid_x"],
           dipoles["base_SdssCentroid_y"],
           dipoles["ip_diffim_DipoleFit_separation"],
           dipoles["ip_diffim_DipoleFit_separation"],
           angles=dipoles["ip_diffim_DipoleFit_orientation"])

In [ ]:
plt.hist(dipoles["ip_diffim_DipoleFit_separation"])
plt.xlabel("Dipole +/- separation [pixels]")

Anything more than 10 pixels is spurious.  Let's take a look at them.

In [ ]:
separation_threshold = 10  # pixels
unbelievably_wide_dipoles = dipoles[dipoles["ip_diffim_DipoleFit_separation"] > separation_threshold].copy(deep=True)

In [ ]:
unbelievably_wide_dipoles["base_PixelFlags_flag"]

In [ ]:
x = unbelievably_wide_dipoles["slot_Centroid_x"]
y = unbelievably_wide_dipoles["slot_Centroid_y"]
print(x, y)

In [ ]:
i = 0
# x, y flipped for imshow
y = int(unbelievably_wide_dipoles["slot_Centroid_x"][i])
x = int(unbelievably_wide_dipoles["slot_Centroid_y"][i])
size = 50  # pixels
small_diff = diff.image.array[x-size:x+size,y-size:y+size]
small_science = science.image.array[x-size:x+size,y-size:y+size]
small_template = template.image.array[x-size:x+size,y-size:y+size]

fig, axes = plt.subplots(1, 3, figsize=(16, 6))
plt.sca(axes[0])
display_diff = afwDisplay.Display(frame=fig)
display_diff.scale("linear", "zscale")
display_diff.mtv(diff)

plt.sca(axes[1])
display_science = afwDisplay.Display(frame=fig)
display_diff.scale("linear", "zscale")
display_diff.mtv(science)

plt.sca(axes[2])
display_template = afwDisplay.Display(frame=fig)
display_template.scale("linear", "zscale")
display_template.mtv(template)


plt.tight_layout()
plt.show()


In [ ]:
SHOW_SCHEMA = False
if SHOW_SCHEMA:
    print(type(dia_sources))
    print(dia_sources.schema)

In [ ]:
STAMP_SIZE = 100
def postage_stamp_from_image(image, x, y, stamp_size=STAMP_SIZE):
    """
    Return postage stamp of stamp_size x stamp_size around given x, y

    x, y can be fractional, but the image will be in original pixels.
    """

    import lsst.geom as geom
    # Naive version that is wrong because it doesn't account for x, y orientation
    center = geom.Point2D(x, y)
    size = geom.Extent2I(stamp_size, stamp_size)
    foo = image.getCutout(center, size)

    return foo

In [ ]:
def show_postage_stamps(*args, figsize=(12, 6)):
    n = len(args)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    displays = []
    for i, image in enumerate(args):
        plt.sca(axes[i])
        display = afwDisplay.Display(frame=fig)
        display.setMaskTransparency(80)
        display.scale("linear", "zscale")
        display.mtv(image)

        displays.append(display)

    plt.tight_layout()
    return fig

In [ ]:
x = unbelievably_wide_dipoles["slot_Centroid_x"]
y = unbelievably_wide_dipoles["slot_Centroid_y"]
for xi, yi in zip(x, y):
    # Show image name?
    d = postage_stamp_from_image(diff, xi, yi)
    s = postage_stamp_from_image(science, xi, yi)
    t = postage_stamp_from_image(template, xi, yi)
    show_postage_stamps(d, s, t);


In [ ]:
display_diff.getMaskPlaneColor()